todo:
- find right lags
- check missing values in id!

In [2]:
import pandas as pd
from feature_engineering import get_features

In [3]:
train = pd.read_csv("train.csv")
train

,id,valeur_NO2,valeur_CO,valeur_O3,valeur_PM10,valeur_PM25
0,2020-01-01 00,42.9,0.718,15.7,73.1,64.4
1,2020-01-01 01,33.6,0.587,10.1,74.8,66.0
2,2020-01-01 02,29.3,NaN,5.1,51.0,44.9
3,2020-01-01 03,30.5,0.246,7.2,27.7,25.1
4,2020-01-01 04,29.3,0.204,8.3,15.3,13.6
...,...,...,...,...,...,...
40986,2024-09-03 18,NaN,0.222,55.1,12.0,5.3
40987,2024-09-03 19,NaN,0.245,48.2,13.4,7.0
40988,2024-09-03 20,NaN,0.234,44.5,12.4,7.1
40989,2024-09-03 21,NaN,0.225,25.9,10.6,5.4


In [4]:
train_new = get_features(train)
train_new

,id,valeur_NO2,valeur_CO,valeur_O3,valeur_PM10,valeur_PM25,year,is_weekend,is_holiday,month_sin,month_cos,weekday_sin,weekday_cos,hour_sin,hour_cos
0,2020-01-01 00:00:00,42.9,0.718,15.7,73.1,64.4,2020,False,True,0.5,8.660254e-01,0.866025,-0.5,0.000000,1.000000
1,2020-01-01 01:00:00,33.6,0.587,10.1,74.8,66.0,2020,False,True,0.5,8.660254e-01,0.866025,-0.5,0.269797,0.962917
2,2020-01-01 02:00:00,29.3,NaN,5.1,51.0,44.9,2020,False,True,0.5,8.660254e-01,0.866025,-0.5,0.519584,0.854419
3,2020-01-01 03:00:00,30.5,0.246,7.2,27.7,25.1,2020,False,True,0.5,8.660254e-01,0.866025,-0.5,0.730836,0.682553
4,2020-01-01 04:00:00,29.3,0.204,8.3,15.3,13.6,2020,False,True,0.5,8.660254e-01,0.866025,-0.5,0.887885,0.460065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40986,2024-09-03 18:00:00,NaN,0.222,55.1,12.0,5.3,2024,False,False,-1.0,-1.836970e-16,0.866025,0.5,-0.979084,0.203456
40987,2024-09-03 19:00:00,NaN,0.245,48.2,13.4,7.0,2024,False,False,-1.0,-1.836970e-16,0.866025,0.5,-0.887885,0.460065
40988,2024-09-03 20:00:00,NaN,0.234,44.5,12.4,7.1,2024,False,False,-1.0,-1.836970e-16,0.866025,0.5,-0.730836,0.682553
40989,2024-09-03 21:00:00,NaN,0.225,25.9,10.6,5.4,2024,False,False,-1.0,-1.836970e-16,0.866025,0.5,-0.519584,0.854419


In [5]:
# check consistency

lagged_date = train_new["id"].shift(1)
diff = (train_new["id"] - lagged_date).dt.total_seconds() / 3600
assert diff.mean() == 1 and diff.std() == 0
diff.describe()

count    40990.0
mean         1.0
std          0.0
min          1.0
25%          1.0
50%          1.0
75%          1.0
max          1.0
Name: id, dtype: float64

In [8]:
train_new.id.describe()

count                  40991
mean     2022-05-03 23:00:00
min      2020-01-01 00:00:00
25%      2021-03-02 23:30:00
50%      2022-05-03 23:00:00
75%      2023-07-04 22:30:00
max      2024-09-03 22:00:00
Name: id, dtype: object

In [5]:
def analyze_missing_values(df):
    """
    Analyze missing values in a DataFrame
    
    Args:
        df: pandas DataFrame to analyze
        
    Returns:
        None (prints analysis results)
    """
    # Calculate missing values summary
    missing_summary = pd.DataFrame({
        'Missing Values': df.isnull().sum(),
        'Percentage Missing': (df.isnull().sum() / len(df) * 100).round(2)
    }).query('`Missing Values` > 0')
    
    # Print overall summary
    print("\nMissing Values Summary:")
    print("-" * 50)
    print(missing_summary)
    
    # If there are any missing values, show patterns
    if missing_summary.shape[0] > 0:
        print("\nMissing Value Patterns:")
        print("-" * 50)
        
        # Show patterns of missingness
        cols_with_missing = df.columns[df.isnull().any()].tolist()
        
        if len(cols_with_missing) > 1:
            # Create correlation matrix of missing values
            missing_corr = df[cols_with_missing].isnull().corr()
            print("\nCorrelation of Missing Values:")
            print(missing_corr)
            
        # Check for time-based patterns if we have datetime
        if 'id' in df.columns and pd.api.types.is_datetime64_any_dtype(df['id']):
            print("\nTemporal Missing Value Patterns:")
            missing_by_time = df[cols_with_missing].isnull().groupby(df['id'].dt.date).mean()
            print(missing_by_time.mean())

## Missing Values Analysis
Let's analyze the missing values in both the original and transformed datasets to understand the data quality.

In [6]:
print("Original Dataset Missing Values:")
analyze_missing_values(train)

print("\nTransformed Dataset Missing Values:")
analyze_missing_values(train_new)

Original Dataset Missing Values:

Missing Values Summary:
--------------------------------------------------
             Missing Values  Percentage Missing
valeur_NO2             3297                8.04
valeur_CO             12529               30.57
valeur_O3               693                1.69
valeur_PM10            7167               17.48
valeur_PM25            1791                4.37

Missing Value Patterns:
--------------------------------------------------

Correlation of Missing Values:
             valeur_NO2  valeur_CO  valeur_O3  valeur_PM10  valeur_PM25
valeur_NO2     1.000000   0.188342   0.144212     0.054918     0.079405
valeur_CO      0.188342   1.000000   0.008701    -0.079832    -0.024980
valeur_O3      0.144212   0.008701   1.000000     0.077137     0.187667
valeur_PM10    0.054918  -0.079832   0.077137     1.000000     0.212987
valeur_PM25    0.079405  -0.024980   0.187667     0.212987     1.000000

Transformed Dataset Missing Values:

Missing Values Summary:
-

In [13]:
import requests, pandas as pd

url = "https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/comptages-routiers-permanents/records"
params = {
 "id_compteur": "A1",
 "date_time": "2024-05-01T10:00:00+00:00",
 "comptage": 125,
 "coordonnees_geo": [48.8566, 2.3522]
}

resp = requests.get(url, params=params).json()
resp
#df = pd.json_normalize(resp['results'])


{'total_count': 26472405,
 'results': [{'iu_ac': '1428',
   'libelle': 'Bd_Mortier',
   't_1h': '2024-09-04T13:00:00+00:00',
   'q': 67.0,
   'k': 1.38222,
   'etat_trafic': 'Fluide',
   'iu_nd_amont': '758',
   'libelle_nd_amont': 'Bd_Mortier-Vidal_de_la_Blache',
   'iu_nd_aval': '757',
   'libelle_nd_aval': 'Bd_Mortier-Av_Pte_Menilmontant',
   'etat_barre': 'Invalide',
   'date_debut': '1996-10-03',
   'date_fin': '2023-01-01',
   'geo_point_2d': {'lon': 2.4089801850993338, 'lat': 48.86985448423951},
   'geo_shape': {'type': 'Feature',
    'geometry': {'coordinates': [[2.4090182436827026, 48.86912730773171],
      [2.408942126515965, 48.87058166074731]],
     'type': 'LineString'},
    'properties': {}}},
  {'iu_ac': '1428',
   'libelle': 'Bd_Mortier',
   't_1h': '2024-09-04T14:00:00+00:00',
   'q': 82.0,
   'k': 1.12556,
   'etat_trafic': 'Fluide',
   'iu_nd_amont': '758',
   'libelle_nd_amont': 'Bd_Mortier-Vidal_de_la_Blache',
   'iu_nd_aval': '757',
   'libelle_nd_aval': 'Bd_Morti

In [16]:
params = {
    "limit": 10,
    "where": "t_1h >= '2021-01-01T00:00:00' AND t_1h < '2022-01-01T00:00:00'"
}

resp = requests.get(url, params=params)
data = resp.json().get("results", [])
print("2021 data available:", len(data) > 0)
if data:
    print("Example record:", data[0])


2021 data available: False
